<center>

# Spatial Accessibility and Social Equity Assessment of Urban Public Facilities: A Case Study in New York City

## Final Project: ST445-Managing and Visualizing Data, 24/25

### Candidates: 45293, 

</center>


## Prerequisites of Notebook

TODO

## Content <br><br>

* [**1. Introduction**](#1)
    * [1.1 Background](#1_1)
    * [1.2 Research Questions and Goals](#1_2)
    * [1.3 Dataset Description](#1_3)
* [**2. Data Acquisiton and Preprocessing**](#2)
    * [2.1 The Boundaries of NYC by Boroughs and Census Tracts](#2_1)
    * [2.2 All Roads and Streets in NYC](#2_2)
    * [2.3 Geographical Points of Public Facilities](#2_3)
    * [2.4 Socio-economic Indicators from ACS](#2_4)
* [**3. Data Visualizations**](#3)
    * [3.1 Spatial Accessibility Distribution of Different Public Facilities](#3_1)
    * [3.2 Equity Evaluation Based on Different Boroughs](#3_2)
    * [3.3 Equity Evaluation Based on Different Population Groups](#3_3)
    * [3.4 Analysis of Contribution Levels of Socio-economic Factors on Accessibility of Facilities](#3_4)
* [**4. Conclusion**](#4)
* [**References**](#5)

<a id="1"></a>
## **1. Introduction**


<a id="1_1"></a>
### 1.1 Background


<a id="1_2"></a>
### 1.2 Research Questions and Goals

<a id="1_3"></a>
### 1.3 Dataset Description

<a id="3"></a>
## **2. Data Acquisiton and Preprocessing**


<a id="2_1"></a>
### 2.1 The Boundaries of NYC by Boroughs and Census Tracts

<a id="2_2"></a>
### 2.2 All Roads and Streets in NYC

<a id="2_3"></a>
### 2.3 Geographical Points of Public Facilities

<a id="2_4"></a>
### 2.4 Socio-economic Indicators from ACS

<a id="3"></a>
## **3. Data Visualizations**


<a id="3_1"></a>
### 3.1 Spatial Accessibility Distribution of Different Public Facilities

<a id="3_2"></a>
### 3.2 Equity Evaluation Based on Different Boroughs

<a id="3_3"></a>
### 3.3 Equity Evaluation Based on Different Population Groups

<a id="3_4"></a>
### 3.4 Analysis of Contribution Levels of Socio-economic Factors on Accessibility of Facilities

<a id="4"></a>
## **4. Conclusion**

<a id="5"></a>
## References